In [1]:
import pandas as pd
import dask.dataframe as dd
from dask_ml import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from joblib import dump

In [2]:
target = ["TARGET", "Target", "target", "CHURN", "Churn", "churn", "RESULT", "Result", "result"]

In [120]:
# Obtiene una lista con las columnas categoricas a partir de una diferencia de conjuntos
def getCategoricalColumns(df):
    cols = set(df.columns)
    numColumns = set(df._get_numeric_data().columns)
    
    return list(cols - numColumns)

# Convierte las variables categoricas en numericas en base a label encoding
def getNumericDataset(df):
    categoricalColumns = getCategoricalColumns(df)
    numDataset = df.drop(columns= categoricalColumns)
    categoricalDataset = df[categoricalColumns]

    le = preprocessing.LabelEncoder()
    for i in categoricalColumns:
        categoricalDataset[i] = le.fit_transform(categoricalDataset[i])
    
    for i in categoricalColumns:
        numDataset[i] = categoricalDataset[i]
    
    return numDataset

In [10]:
def scaler(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

In [3]:
def reduce_csv(csv_file):
    df = dd.read_csv(csv_file)
    result = df[df.columns.intersection(target)]
    df = df.drop(columns= target + ["Unnamed: 0"], errors='ignore')
    
    x = getNumericDataset(df)
    columns = list(x.columns)
    x = scaler(x)
    
    pca = PCA(n_components=.8, svd_solver='full')
    pca.fit(x)

    reduced = []
    for i in range(0, len(pca.explained_variance_ratio_)):
        reduced.append(columns[i])   

    result[reduced] = df[reduced]

    result.to_csv(csv_file, index=False, single_file=True)


In [8]:
csv_file = "/home/alt9193/Documents/IA/DeepLearningBackend/examples/telecom_churn_me copy.csv"

In [126]:
reduce_csv(csv_file)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict

In [5]:
def create_model(csv_file, model_file):
    df = dd.read_csv(csv_file)
    y = df[df.columns.intersection(target)].to_dask_array(lengths=True)
    x = df.drop(columns= target + ["Unnamed: 0"], errors='ignore')
    x = dd.from_array(scaler(df)).to_dask_array(lengths=True)

    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.25, random_state= 1)

    random_forest = RandomForestClassifier(max_depth = 3, random_state = 1)

    kfold = KFold(n_splits = 5, random_state=42, shuffle=True)
    cv_results = cross_val_score(random_forest, x,y, cv = kfold, scoring='accuracy', verbose = 0)

    random_forest = random_forest.fit(x_train, y_train)

    dump(random_forest, model_file)
    return x

In [6]:
from joblib import dump

In [11]:
x = create_model(csv_file, 'random_forest_churn.joblib')

/home/alt9193/.local/lib/python3.10/site-packages/sklearn/utils/__init__.py:185: PerformanceWarning: Slicing with an out-of-order index is generating 35562 times more chunks
  return array[key] if axis == 0 else array[:, key]
/home/alt9193/.local/lib/python3.10/site-packages/sklearn/utils/__init__.py:185: PerformanceWarning: Slicing with an out-of-order index is generating 11858 times more chunks
  return array[key] if axis == 0 else array[:, key]
/home/alt9193/.local/lib/python3.10/site-packages/sklearn/utils/__init__.py:185: PerformanceWarning: Slicing with an out-of-order index is generating 214184 times more chunks
  return array[key] if axis == 0 else array[:, key]
/home/alt9193/.local/lib/python3.10/site-packages/sklearn/utils/__init__.py:185: PerformanceWarning: Slicing with an out-of-order index is generating 71177 times more chunks
  return array[key] if axis == 0 else array[:, key]
/home/alt9193/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: D

KeyboardInterrupt: 